In [1]:
import sys, os
import glob

repo_root = os.path.abspath("..")
sys.path.append(repo_root)

from MyClasses import PONE_Reader, I3FeatureExtractorPONE, I3TruthExtractorPONE

graphnet [MainProcess] WARNING  2026-01-03 14:08:16 - <module> - `km3net` not available. Some functionality may be missing.


/home/kbas/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


graphnet [MainProcess] WARNING  2026-01-03 14:12:31 - <module> - `jammy_flows` not available. Normalizing Flow functionality is missing.


In [2]:
from graphnet.data.dataconverter import DataConverter


In [3]:
from graphnet.data.dataconverter import DataConverter
from graphnet.data.writers import ParquetWriter
from graphnet.data.extractors.icecube.utilities.i3_filters import NullSplitI3Filter

In [4]:
# INPUT_DIR = "/project/def-nahee/kbas/POM_Response"
GCD_RESCUE = "/project/6008051/pone_simulation/GCD_Library/PONE_800mGrid.i3.gz"
OUTDIR = "/project/def-nahee/kbas/POM_Response_Parquet"

In [16]:
INPUT_GLOB = "/project/def-nahee/kbas/POM_Response_GZ/*.i3.gz"
N_FILES = 20  # deneme için 1-2 yap

In [17]:
files = sorted(glob.glob(INPUT_GLOB))[:N_FILES]
print("Converting these files:\n", "\n".join(files))


Converting these files:
 /project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_100.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_1000.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_1001.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_1002.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_1003.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_1004.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_1005.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_1006.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_1007.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_1008.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_1009.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_101.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_1010.i3.gz
/project/def-nahee/kbas/POM_Response_GZ/pom_response_batch_1011.i3.

In [18]:
reader = PONE_Reader(
    gcd_rescue=GCD_RESCUE,
    i3_filters=NullSplitI3Filter(),
)

In [19]:
reader = PONE_Reader(
    gcd_rescue=GCD_RESCUE,
    i3_filters=NullSplitI3Filter(),
)

In [20]:
extractors = [
    I3FeatureExtractorPONE(pulsemap="EventPulseSeries", name="features", exclude=['pmt_area', 'rde', 'width', 'event_time', 'is_bright_dom', 'is_saturated_dom', 'is_errata_dom', 'is_bad_dom', 'hlc','awtd','string', 'pmt_number', 'dom_number', 'dom_type']),
    I3TruthExtractorPONE(mctree="I3MCTree_postprop", name="truth", exclude=['L7_oscNext_bool', 'L6_oscNext_bool',
                                                               'L5_oscNext_bool', 'L4_oscNext_bool',
                                                               'L3_oscNext_bool',
                                                               'OnlineL2Filter_17','MuonFilter_13',
                                                               'CascadeFilter_13','DeepCoreFilter_13', 
                                                                'dbang_decay_length', 'track_length', 'stopped_muon', 'energy_track', 'energy_cascade', 'inelasticity', 'is_starting'] ),

]


In [21]:
writer = ParquetWriter(truth_table="truth", index_column="event_no")

In [22]:
converter = DataConverter(
    file_reader=reader,
    save_method=writer,  
    extractors=extractors,
    outdir=OUTDIR,
    num_workers=1,
    index_column="event_no",
)

In [23]:
converter(input_dir=files)

Assuming list of files.
graphnet [MainProcess] INFO     2026-01-03 14:13:06 - DataConverter.__call__ - Processing 20 file(s) in main thread(not multiprocessing)


100%|█████████████████████████████████████| 20/20 [03:13<00:00,  9.68s/ file(s)]


In [24]:
print("DONE:", OUTDIR)

DONE: /project/def-nahee/kbas/POM_Response_Parquet


In [25]:
MERGED_DIR = os.path.join(OUTDIR, "merged")


In [26]:
writer.merge_files(
    files=[],                
    output_dir=MERGED_DIR,   # <-- burasi OUTDIR/merged olacak
    events_per_batch=200000, # istersen 50000 yap
    num_workers=1,
)

graphnet [MainProcess] INFO     2026-01-03 14:16:40 - ParquetWriter.run_code - Processing 1 batches in main thread.


100%|███████████████████████████████████████| 1/1 [00:01<00:00,  1.60s/shard(s)]
